In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


#<LIBRERIAS>

#Para manipular y análisar datos
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv)

#Es un metaestimador que se ajusta a varios clasificadores de árboles de decisión
#en varias submuestras del conjunto de datos y usa promedios para mejorar la precisión
#predictiva y controlar el sobreajuste.
from sklearn.ensemble import RandomForestClassifier

#Implementa el algoritmo Gaussiano Naive Bayes para datos distribuidos multinomialmente.
from sklearn.naive_bayes import GaussianNB

#Evaluar una puntuación mediante validación cruzada
from sklearn.model_selection import cross_val_score

#Proporciona índices de entrenamiento / prueba
#para dividir datos en conjuntos random
from sklearn.model_selection import StratifiedShuffleSplit

#Red neuronal
from sklearn.neural_network import MLPClassifier

#Arbol de decision
from sklearn import tree

#</LIBRERIAS>


#importar data-set
data = pd.read_csv("/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# DATA CLEANING

*Verificamos la lectura de los datos*

In [ ]:
data.head()

*Contar valores perdidos en columna*

In [ ]:
data.isnull().sum()

*Podemos trabajar con los datos*

media, varianza, min, max, ...

In [ ]:
data.describe()

Para los datos etiquetados X: age, anaemia, creatinine_phosphokinase, diabetes, ejection_fraction, high_blood_pressure, platelets, serum_creatinine, serum_sodium, sex, smoking
descartamos time, DEATH_EVENT

In [ ]:
X=data.iloc[:,:-2]
X.head()

Para nuestra clase no etiquetada y: Columna de EVENTO DE MUERTE

In [ ]:
y=data.DEATH_EVENT
y.head()

# PROCESAMIENTO DE DATOS

*Proporciona indices de entrenamiento/prueba random para dividir datos en entrenamiento/prueba; splits=10  ;evaluar 10 veces consecutivas(con diferentes divisiones); tamaño de prueba 20%*

In [ ]:
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.20, random_state=True)

*Genere índices para dividir los datos en conjuntos de entrenamiento y prueba*

In [ ]:
iter_for_prediction = cv.split(X, y)

In [ ]:
#columna indice, (conjunto de indices de entrenamiento, conjunto de indices de prueba)
for index, (train_index, test_index) in enumerate(iter_for_prediction):
    #número de split
    print(index)

    #RandomForestClassifier
    #Es un metaestimador que se ajusta a varios clasificadores de árboles de decisión
    #en varias submuestras del conjunto de datos y usa promedios para mejorar la precisión
    #predictiva y controlar el sobreajuste.

    #Controla tanto la aleatoriedad del bootstrapping de las muestras utilizadas
    #al construir árboles.
    #Utilice un nuevo generador de números aleatorios sembrado por el entero dado.
    #El uso de un int producirá los mismos resultados en diferentes llamadas.
    #Sin embargo, puede valer la pena comprobar que los resultados sean estables
    #en varias semillas aleatorias distintas. 
    randomf = RandomForestClassifier(random_state=0)
    
    #Implementa el algoritmo Gaussiano Naive Bayes para datos distribuidos multinominalmente.
    gnb = GaussianNB()

    #>>>MLP Classifier[Red neuronal]<<<
    #"Lbfgs" es un optimizador basado en métodos cuasi-Newtonianos.
    #"Logistic", la función sigmoidea logística, devuelve f (x) = 1 / (1 + exp (-x)).
    #"verbose", imprimir mensajes de progreso en stdout.
    #"alpha", L2 penalty (regularization term) parameter.
    #"hidden_layer_sizes", El i-ésimo elemento representa el número de neuronas en la i-ésima capa oculta.
    mlp=MLPClassifier(solver = 'lbfgs', activation='logistic', verbose=False, alpha=1e-4, tol = 1e-3, hidden_layer_sizes=(150, 2))


    #Arbol de decision
    arbol = tree.DecisionTreeClassifier(random_state=0)

    #Regresion lineal
    #lr = linear_model.LinearRegression()
    #no puedo usarlo porque mi salida es boolean


    sk_clasificador = {'GaussianNB':gnb,
                       'RandomForest':randomf,
                       'MLPCassifier': mlp,
                       'Arbol': arbol}
                       #'lineal': lr}
    #i=1

    #itera entre GaussianNB y RandomForest
    for clasifier in sk_clasificador:
        print(clasifier)
        #print(i, ": ", clasifier)
        #i = i+1

        estimador = sk_clasificador[clasifier]

        #Validación cruzada: evaluación del desempeño del estimador 
        #estimador: objeto estimador que implementa 'ajuste'. El objeto que se utilizará para ajustar los datos.
        #X: dato a ajustar
        #y: La variable objetivo para intentar predecir en el caso del aprendizaje supervisado.

        #scoring: exactitud
        score = cross_val_score(estimador, X, y, scoring='accuracy', cv=[(train_index, test_index)])
        print("SCORE")
        #print(estimador, score) #score: numpy.ndarray
        print(score)